In [2]:
from IPython.display import clear_output

import datetime
import time
import os


from tools import (
    play_beep,
    print_account_summary,
    print_order,
    print_openorders,
)

from IPython.display import display
from IPython.core.display import HTML

play_beep()

display(HTML("<style>.output_subarea { overflow: auto; }</style>"))

import socket

hostname = socket.gethostname()


In [4]:
from ib_async import *

util.startLoop()

ib = IB()
ib.connect("127.0.0.1", 4001, clientId=13)
util.logToFile(f"{datetime.datetime.now().strftime('%Y-%m-%d')}-{hostname}.log")


open orders request timed out
completed orders request timed out
account updates for U10394496 request timed out
account updates for U14726869 request timed out
account updates for U2340948 request timed out
completed orders request timed out
account updates for U10394496 request timed out
account updates for U14726869 request timed out
account updates for U2340948 request timed out
executions request timed out


In [4]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(index=range(5), columns="bidSize bidPrice askPrice askSize".split())

# NQM2024 contract
contract = Contract(conId=620730920)
ib.qualifyContracts(contract)

ticker = ib.reqMktDepth(contract)


def onTickerUpdate(ticker):
    bids = ticker.domBids
    for i in range(5):
        df.iloc[i, 0] = bids[i].size if i < len(bids) else 0
        df.iloc[i, 1] = bids[i].price if i < len(bids) else 0
    asks = ticker.domAsks
    for i in range(5):
        df.iloc[i, 2] = asks[i].price if i < len(asks) else 0
        df.iloc[i, 3] = asks[i].size if i < len(asks) else 0
    clear_output(wait=True)
    display(df)


# ticker.updateEvent += onTickerUpdate

# IB.sleep(15)

# ib.cancelMktDepth(contract)


In [9]:
# default reset values
open_trade = None
close_trade = None
# get a specific order by id

# trades = get_all_openorders(ib)
# open_trade = [trade for trade in trades if trade.order.permId == 449873795]
# print_order(open_trade)

# future = [
#     pos for pos in ib.positions(account="U10394496") if pos.contract.symbol == "NQ"
# ][0]

# print(future)
print()
print_account_summary(ib=ib)
print()
print_openorders(ib=ib)



AccountType : INDIVIDUAL
Cushion : 0.533765
LookAheadNextChange : 1714766400
AccruedCash : 0.00
AvailableFunds : 96011.15
BuyingPower : 418345.99
EquityWithLoanValue : 109249.29
ExcessLiquidity : 105500.14
FullAvailableFunds : 55584.52
FullExcessLiquidity : 68748.65
FullInitMarginReq : 139418.23
FullMaintMarginReq : 126254.10
GrossPositionValue : 2911.98
InitMarginReq : 98991.60
LookAheadAvailableFunds : 55584.52
LookAheadExcessLiquidity : 68748.65
LookAheadInitMarginReq : 139418.23
LookAheadMaintMarginReq : 126254.10
MaintMarginReq : 89502.62
NetLiquidation : 197652.74
PreviousDayEquityWithLoanValue : 4266.00
RegTEquity : 5046.96
RegTMargin : 3749.15
SMA : 624735.50
TotalCashValue : 195002.75

742712408		SELL	1.0	18001.0	


In [13]:
# t1 = get_all_openorders(ib)[1]
# print(t1)

open_trade = None
close_trade = None

util.df(ib.openOrders())


,orderId,clientId,permId,action,totalQuantity,orderType,lmtPrice,auxPrice,tif,activeStartTime,...,usePriceMgmtAlgo,duration,postToAts,advancedErrorOverride,manualOrderTime,minTradeQty,minCompeteSize,competeAgainstBestOffset,midOffsetAtWhole,midOffsetAtHalf
0,367,13,742712424,BUY,1.0,LMT,17970.75,0.0,,,...,False,2147483647,2147483647,,,2147483647,2147483647,1.797693e+308,1.797693e+308,1.797693e+308


In [11]:
## SELL TO OPEN SCALP 0.1
########################################

sell_scalp = {
    "strategy": "SELL TO OPEN SCALP",
    "contract": "NQM2024",
    "tick_increment": 0.25,
    "open_qty": 1,
    "open_type": "LIMIT",
    "open_action": "SELL",
    "open_ref": "ask",
    "open_ticks": 10,
    "close_qty": 1,
    "close_type": "LIMIT",
    "close_action": "BUY",
    "close_ref": "open_price_fill",
    "close_ticks": -10,
    "pause_seconds": 30,
}

buy_scalp = {
    "strategy": "BUY TO OPEN SCALP",
    "contract": "NQM2024",
    "tick_increment": 0.25,
    "open_qty": 1,
    "open_type": "LIMIT",
    "open_action": "BUY",
    "open_ref": "bid",
    "open_ticks": -20,
    "close_qty": 1,
    "close_type": "LIMIT",
    "close_action": "SELL",
    "close_ref": "open_price_fill",
    "close_ticks": 10,
    "pause_seconds": 60,
}


strategy_details = sell_scalp

print(strategy_details["strategy"])

while True:
    clear_output(wait=True)

    # first order of the strategy
    if open_trade is None and close_trade is None:
        action = strategy_details["open_action"]
        qty = strategy_details["open_qty"]

        if strategy_details["open_ref"] == "bid":
            price_ref = ticker.domBids[0].price
        elif strategy_details["open_ref"] == "ask":
            price_ref = ticker.domAsks[0].price
        elif strategy_details["open_ref"] == "mid":
            price_ref = (ticker.domAsks[0].price + ticker.domBids[0].price) / 2
        elif strategy_details["open_ref"] == "last":
            raise Exception("Not implemented")

        lmtPrice = (
            price_ref
            + strategy_details["open_ticks"] * strategy_details["tick_increment"]
        )
        print(
            f"Placing open trade: {action}, {strategy_details['open_type']}, totalQuantity {qty}, lmtPrice {lmtPrice}\n"
        )
        print()

        if strategy_details["open_type"] == "LIMIT":
            open_order = LimitOrder(
                action=action,
                totalQuantity=qty,
                lmtPrice=lmtPrice,
                account="U10394496",
            )
        else:
            raise Exception("Not implemented")

        open_trade = ib.placeOrder(contract, open_order)
        open_order_ts = datetime.datetime.now()

    print("OPEN ORDER::")
    print_order(open_trade)
    print()

    if open_trade is not None:
        if open_trade.orderStatus.status == "Submitted" and close_trade is None:
            print(
                f"Waiting to get filled on order #{open_trade.order.permId} ({open_trade.orderStatus.status})\n"
            )

            if datetime.datetime.now() - open_order_ts > datetime.timedelta(
                seconds=strategy_details["pause_seconds"]
            ):
                print("Cancelling order due to timeout:")
                ib.cancelOrder(open_trade.order)
                ib.sleep(1)
                print()

        if open_trade.orderStatus.status == "Filled" and close_trade is None:
            action = strategy_details["close_action"]
            qty = strategy_details["close_qty"]

            if strategy_details["close_ref"] == "open_price_fill":
                price_ref = open_trade.orderStatus.avgFillPrice
            if strategy_details["close_ref"] == "bid":
                price_ref = ticker.domBids[0].price
            elif strategy_details["close_ref"] == "ask":
                price_ref = ticker.domAsks[0].price
            elif strategy_details["close_ref"] == "mid":
                price_ref = (ticker.domAsks[0].price + ticker.domBids[0].price) / 2
            elif strategy_details["close_ref"] == "last":
                raise Exception("Not implemented")

            lmtPrice = (
                price_ref
                + strategy_details["close_ticks"] * strategy_details["tick_increment"]
            )

            if strategy_details["close_type"] == "LIMIT":
                close_order = LimitOrder(
                    action=action,
                    totalQuantity=qty,
                    lmtPrice=lmtPrice,
                    account="U10394496",
                )
            else:
                raise Exception("Not implemented")

            close_trade = ib.placeOrder(contract, close_order)
            ib.sleep(1)
            play_beep(500, 500)

        elif (
            open_trade.orderStatus.status == "Inactive"
            or open_trade.orderStatus.status == "Cancelled"
        ) and close_trade is None:
            print("***** order is inactive *****")
            print(open_trade.log)
            print("*****************************")
            open_trade = None

    print(f"CLOSE ORDER::")
    print_order(close_trade)
    print()

    if close_trade is not None:
        if close_trade.orderStatus.status == "Filled":
            play_beep(2500, 500)
            print(
                "Close trade filled @ {}\n".format(close_trade.orderStatus.avgFillPrice)
            )
            open_trade = None
            close_trade = None
            ib.accountSummary()

            # small break before next trade
            for i in range(5):
                print(".", end="")
                ib.sleep(1)

    print("ALL OPEN ORDERS::")
    print_openorders(ib=ib)
    print()

    if ticker is not None and ticker.domBids is not None and ticker.domAsks is not None:
        for i in range(min(len(ticker.domBids), len(ticker.domAsks))):
            bid_size = ticker.domBids[i].size
            bid_price = ticker.domBids[i].price
            ask_price = ticker.domAsks[i].price
            ask_size = ticker.domAsks[i].size
            print(f"{bid_size:>8} {bid_price:>10} | {ask_price:<10} {ask_size:<8}")

    print()
    fills = [t.fills for t in ib.trades() if t.fills != []]
    executions = [f[0].execution for f in fills]
    print(f"Day Executions: {len(executions)}")

    future = [pos for pos in ib.positions() if pos.contract.symbol == "NQ"]
    for f in future:
        print(
            f"{f.contract.symbol} {f.position} @ {f.avgCost/float(contract.multiplier)}"
        )
    print()
    print_account_summary(ib=ib)
    ib.sleep(3)


OPEN ORDER::
symbol	permId		status		action	filled	remaining	lmtPrice
NQ	742712423	Filled	SELL	1.0	0.0		17973.25	

CLOSE ORDER::
symbol	permId		status		action	filled	remaining	lmtPrice
NQ	742712424	Submitted	BUY	0.0	1.0		17970.75	

ALL OPEN ORDERS::
742712424		BUY	1.0	17970.75	

     1.0   18012.25 | 18012.75   3.0     
     6.0    18012.0 | 18013.0    6.0     
     6.0   18011.75 | 18013.25   3.0     
     8.0    18011.5 | 18013.5    4.0     
     8.0   18011.25 | 18013.75   7.0     

Day Executions: 120
NQ -7.0 @ 17985.3884179225

AccountType : INDIVIDUAL
Cushion : 0.351335
LookAheadNextChange : 1714766400
AccruedCash : 0.00
AvailableFunds : 55581.26
BuyingPower : 270388.76
EquityWithLoanValue : 72360.26
ExcessLiquidity : 68550.80
FullAvailableFunds : -1065.24
FullExcessLiquidity : 17053.97
FullInitMarginReq : 193584.75
FullMaintMarginReq : 175465.54
GrossPositionValue : 2950.02
InitMarginReq : 136938.25
LookAheadAvailableFunds : -1065.24
LookAheadExcessLiquidity : 17053.97
LookAheadI

KeyboardInterrupt: 

In [ ]:
fills = [t.fills for t in ib.trades() if t.fills != []]
executions = [f[0].execution for f in fills]
util.df(executions)


In [8]:
ib.cancelOrder(open_trade.order)


Trade(contract=Contract(secType='FUT', conId=620730920, symbol='NQ', lastTradeDateOrContractMonth='20240621', multiplier='20', exchange='CME', currency='USD', localSymbol='NQM4', tradingClass='NQ'), order=LimitOrder(orderId=190, clientId=13, permId=742712184, action='SELL', totalQuantity=1.0, lmtPrice=17952.0, auxPrice=0.0, account='U10394496'), orderStatus=OrderStatus(orderId=190, status='PendingCancel', filled=0.0, remaining=1.0, avgFillPrice=0.0, permId=742712184, parentId=0, lastFillPrice=0.0, clientId=13, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 5, 3, 13, 37, 39, 879350, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 5, 3, 13, 37, 39, 935407, tzinfo=datetime.timezone.utc), status='Submitted', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 5, 3, 13, 38, 28, 783669, tzinfo=datetime.timezone.utc), status='PendingCancel', message='', errorCo

In [ ]:
print_all_openorders(ib=ib)


In [ ]:
trades = ib.reqAllOpenOrders()

trades.sort(key=lambda trade: trade.order.lmtPrice)

for trade in trades:
    orderstatus = trade.orderStatus
    order = trade.order

    if trade.contract.symbol != "NQ":
        continue

    print(
        f"{trade.contract.symbol}\t{order.permId}\t{orderstatus.status}\t{order.action}\t{orderstatus.filled}\t{orderstatus.remaining}\t\t{order.lmtPrice}\t"
    )


In [ ]:
trades_df = util.df(ib.trades())
trades_df


In [ ]:
import sqlite3

# Create a connection to the SQLite database
conn = sqlite3.connect("trades.db")

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Create the "trades" table if it doesn't exist
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS trades (
        contract TEXT,
        order TEXT,
        orderStatus TEXT,
        fills TEXT,
        log TEXT,
        advancedError TEXT
    )
"""
)

# Convert the DataFrame to a list of tuples
trades_data = trades_df.to_records(index=False).tolist()

# Insert the data into the "trades" table
cursor.executemany(
    """
    INSERT INTO trades (contract, order, orderStatus, fills, log, advancedError)
    VALUES (?, ?, ?, ?, ?, ?)
""",
    trades_data,
)

# Commit the changes and close the connection
conn.commit()
conn.close()


In [ ]:
# get all the trades where fills is not []
fills = trades_df[trades_df.fills.map(len) > 0]

fills_list = list(fills["order"])

# loop through fills and convert each member of this data type to a row in a df
#  Order(permId=253828901, action='BUY', orderType='LMT', lmtPrice=17580.0, auxPrice=0.0, tif='GTC', ocaType=3, displaySize=2147483647, rule80A='0', openClose='', volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U10394496', clearingIntent='IB', cashQty=0.0, dontUseAutoPriceForHedge=True, autoCancelDate='20240930 16:00:00 Central Standard Time', filledQuantity=1.0, refFuturesConId=2147483647, shareholder='Not an insider or substantial shareholder')
